### Student Information
Name: 蔡秉翰

Student ID: 107064527

GitHub ID: k52276hank

Kaggle name: Pin-Han, Tsai

Kaggle private scoreboard snapshot:

![Snapshot](img/pic0.png)

---

### Instructions

3. Third: __This part is worth 30% of your grade.__ A report of your work developping the model for the competition (You can use code and comment it). This report should include what your preprocessing steps, the feature engineering steps and an explanation of your model. You can also mention different things you tried and insights you gained. 

---

# Import Package for Preprocessing

In [1]:
# import related library
import pandas as pd
import numpy as np
import json
import re, csv
import nltk
import string

from pandas.io.json import json_normalize
from nltk.corpus import stopwords
from collections import defaultdict
from nltk.corpus import wordnet
from nltk import pos_tag
from nltk.stem import WordNetLemmatizer


# Transform Raw Data to DataFrame

## Read Identification Data to DataFrame

In [2]:
# load csv and restructure dataframe (rename)
df_iden = pd.read_csv("data/data_identification.csv", sep="\t")
df_iden['tweet_id'] = df_iden['tweet_id,identification'].str.split(',', expand=True)[0]
df_iden['identification'] = df_iden['tweet_id,identification'].str.split(',', expand=True)[1]
df_iden = df_iden.drop('tweet_id,identification', axis=1)
df_iden.head()

,tweet_id,identification
0,0x28cc61,test
1,0x29e452,train
2,0x2b3819,train
3,0x2db41f,test
4,0x2a2acc,train


## Read Emotion Data to DataFrame

In [3]:
df_emotion = pd.read_csv("data/emotion.csv", sep="\t")
df_emotion['tweet_id'] = df_emotion['tweet_id,emotion'].str.split(',', expand=True)[0]
df_emotion['emotion'] = df_emotion['tweet_id,emotion'].str.split(',', expand=True)[1]
df_emotion = df_emotion.drop('tweet_id,emotion', axis=1)
df_emotion.head()

,tweet_id,emotion
0,0x3140b1,sadness
1,0x368b73,disgust
2,0x296183,anticipation
3,0x2bd6e1,joy
4,0x2ee1dd,anticipation


## Find Duplicated of DataFrame

In [4]:
# check if duplicated
df_emotion.duplicated(keep='first')
df_iden.duplicated(keep='first')

0          False
1          False
2          False
3          False
4          False
5          False
6          False
7          False
8          False
9          False
10         False
11         False
12         False
13         False
14         False
15         False
16         False
17         False
18         False
19         False
20         False
21         False
22         False
23         False
24         False
25         False
26         False
27         False
28         False
29         False
           ...  
1867505    False
1867506    False
1867507    False
1867508    False
1867509    False
1867510    False
1867511    False
1867512    False
1867513    False
1867514    False
1867515    False
1867516    False
1867517    False
1867518    False
1867519    False
1867520    False
1867521    False
1867522    False
1867523    False
1867524    False
1867525    False
1867526    False
1867527    False
1867528    False
1867529    False
1867530    False
1867531    False
1867532    Fal

## Read All Raw Tweet Data

In [5]:
# load raw data of twitter
total_datas = open('data/tweets_DM.json').read()
tweet_DM = pd.read_json(total_datas, lines = True)
tweet_DM.head()

,_crawldate,_index,_score,_source,_type
0,2015-05-23 11:42:47,hashtag_tweets,391,"{'tweet': {'hashtags': ['Snapchat'], 'text': '...",tweets
1,2016-01-28 04:52:09,hashtag_tweets,433,"{'tweet': {'hashtags': ['freepress', 'TrumpLeg...",tweets
2,2017-12-25 04:39:20,hashtag_tweets,232,"{'tweet': {'hashtags': ['bibleverse'], 'text':...",tweets
3,2016-01-24 23:53:05,hashtag_tweets,376,"{'tweet': {'hashtags': [], 'text': 'Now ISSA i...",tweets
4,2016-01-08 17:18:59,hashtag_tweets,989,"{'tweet': {'hashtags': [], 'text': '""Trust is ...",tweets


## Convert to DataFrame

In [6]:
# get some useful information from all data
df_total = json_normalize(tweet_DM['_source'])
df_total.rename(columns={'tweet.tweet_id':'tweet_id', 
                 'tweet.text':'text', 
                 'tweet.hashtags': 'hashtags'}, inplace=True)
df_total.head()

,hashtags,text,tweet_id
0,[Snapchat],"People who post ""add me on #Snapchat"" must be ...",0x376b20
1,"[freepress, TrumpLegacy, CNN]","@brianklaas As we see, Trump is dangerous to #...",0x2d5350
2,[bibleverse],"Confident of your obedience, I write to you, k...",0x28b412
3,[],Now ISSA is stalking Tasha 😂😂😂 <LH>,0x1cd5b0
4,[],"""Trust is not the same as faith. A friend is s...",0x2de201


## Merge Total Tweet Data and Emotion and Identification

In [7]:
# merge data sets
df_total_merge = pd.merge(df_total, df_emotion, how='outer', on=['tweet_id'])
df_total_merge = pd.merge(df_total_merge, df_iden, how='outer', on=['tweet_id'])
df_total_merge.head()

,hashtags,text,tweet_id,emotion,identification
0,[Snapchat],"People who post ""add me on #Snapchat"" must be ...",0x376b20,anticipation,train
1,"[freepress, TrumpLegacy, CNN]","@brianklaas As we see, Trump is dangerous to #...",0x2d5350,sadness,train
2,[bibleverse],"Confident of your obedience, I write to you, k...",0x28b412,NaN,test
3,[],Now ISSA is stalking Tasha 😂😂😂 <LH>,0x1cd5b0,fear,train
4,[],"""Trust is not the same as faith. A friend is s...",0x2de201,NaN,test


## Seperate Train DataFrame and Test DataFrame

In [8]:
# seperate training and testing data
train_df = df_total_merge[df_total_merge["identification"]=="train"]
test_df = df_total_merge[df_total_merge["identification"]=="test"]

In [9]:
train_df.head()

,hashtags,text,tweet_id,emotion,identification
0,[Snapchat],"People who post ""add me on #Snapchat"" must be ...",0x376b20,anticipation,train
1,"[freepress, TrumpLegacy, CNN]","@brianklaas As we see, Trump is dangerous to #...",0x2d5350,sadness,train
3,[],Now ISSA is stalking Tasha 😂😂😂 <LH>,0x1cd5b0,fear,train
5,"[authentic, LaughOutLoud]",@RISKshow @TheKevinAllison Thx for the BEST TI...,0x1d755c,joy,train
6,[],Still waiting on those supplies Liscus. <LH>,0x2c91a8,anticipation,train


In [10]:
test_df.head()

,hashtags,text,tweet_id,emotion,identification
2,[bibleverse],"Confident of your obedience, I write to you, k...",0x28b412,NaN,test
4,[],"""Trust is not the same as faith. A friend is s...",0x2de201,NaN,test
9,"[materialism, money, possessions]",When do you have enough ? When are you satisfi...,0x218443,NaN,test
30,"[GodsPlan, GodsWork]","God woke you up, now chase the day #GodsPlan #...",0x2939d5,NaN,test
33,[],"In these tough times, who do YOU turn to as yo...",0x26289a,NaN,test


## Save to Pickle before Preprocessing

In [11]:
# save to pickle file
train_df.to_pickle("data/train_df_nonpreprocess.pkl") 
test_df.to_pickle("data/test_df_nonpreprocess.pkl")

## Read Pickle which is Non-Preprocessing

In [12]:
# load a pickle file
train_df = pd.read_pickle("data/train_df_nonpreprocess.pkl")
test_df = pd.read_pickle("data/test_df_nonpreprocess.pkl")

# Preprocessing

## Tokenize at first without filter anything

In [13]:
df_input = train_df
df_input["split_to_word"] = df_input["text"].str.split()
df_input.head()

,hashtags,text,tweet_id,emotion,identification,split_to_word
0,[Snapchat],"People who post ""add me on #Snapchat"" must be ...",0x376b20,anticipation,train,"[People, who, post, ""add, me, on, #Snapchat"", ..."
1,"[freepress, TrumpLegacy, CNN]","@brianklaas As we see, Trump is dangerous to #...",0x2d5350,sadness,train,"[@brianklaas, As, we, see,, Trump, is, dangero..."
3,[],Now ISSA is stalking Tasha 😂😂😂 <LH>,0x1cd5b0,fear,train,"[Now, ISSA, is, stalking, Tasha, 😂😂😂, <LH>]"
5,"[authentic, LaughOutLoud]",@RISKshow @TheKevinAllison Thx for the BEST TI...,0x1d755c,joy,train,"[@RISKshow, @TheKevinAllison, Thx, for, the, B..."
6,[],Still waiting on those supplies Liscus. <LH>,0x2c91a8,anticipation,train,"[Still, waiting, on, those, supplies, Liscus.,..."


## I. Replace @tag to [tag]

In [14]:
df_input["replace_tag"] = df_input.split_to_word.apply(
    lambda list_word: 
    ["[tag]" if word[0]=='@' else word for word in list_word]
)
df_input.head()

,hashtags,text,tweet_id,emotion,identification,split_to_word,replace_tag
0,[Snapchat],"People who post ""add me on #Snapchat"" must be ...",0x376b20,anticipation,train,"[People, who, post, ""add, me, on, #Snapchat"", ...","[People, who, post, ""add, me, on, #Snapchat"", ..."
1,"[freepress, TrumpLegacy, CNN]","@brianklaas As we see, Trump is dangerous to #...",0x2d5350,sadness,train,"[@brianklaas, As, we, see,, Trump, is, dangero...","[[tag], As, we, see,, Trump, is, dangerous, to..."
3,[],Now ISSA is stalking Tasha 😂😂😂 <LH>,0x1cd5b0,fear,train,"[Now, ISSA, is, stalking, Tasha, 😂😂😂, <LH>]","[Now, ISSA, is, stalking, Tasha, 😂😂😂, <LH>]"
5,"[authentic, LaughOutLoud]",@RISKshow @TheKevinAllison Thx for the BEST TI...,0x1d755c,joy,train,"[@RISKshow, @TheKevinAllison, Thx, for, the, B...","[[tag], [tag], Thx, for, the, BEST, TIME, toni..."
6,[],Still waiting on those supplies Liscus. <LH>,0x2c91a8,anticipation,train,"[Still, waiting, on, those, supplies, Liscus.,...","[Still, waiting, on, those, supplies, Liscus.,..."


## II. Replace emoji to word

### setup emoji to emotion dictionary

In [16]:
from data.unicode_codes import EMOJI_UNICODE, UNICODE_EMOJI

list_cal_emoji = df_input.replace_tag.apply(
    lambda list_word: 
    [char for char in " ".join(list_word) if char in UNICODE_EMOJI]
)

emotion_array = ['joy', 'sadness', 'trust', 'anticipation', 'fear', 'anger', 'disgust', 'surprise']
dict_emoji_all_emotion = defaultdict(list)
dict_emoji_emotion = {}
seties_emotion = df_input.emotion
for idx, list_emoji in enumerate(list_cal_emoji): 
    list_emoji = list(set(list_emoji))  
    for emoji in list_emoji:
        if not dict_emoji_all_emotion[emoji]: dict_emoji_all_emotion[emoji]=[0]*len(emotion_array)
        dict_emoji_all_emotion[emoji][emotion_array.index(seties_emotion.iloc[idx])] += 1

for emoji, list_emoji_num in dict_emoji_all_emotion.items():
    dict_emoji_emotion[emoji] = emotion_array[np.argmax(list_emoji_num)]

dict_emoji_emotion

{'😄': 'joy',
 '😛': 'joy',
 '😗': 'joy',
 '🌮': 'anticipation',
 '🍗': 'joy',
 '🏡': 'joy',
 '🗒': 'joy',
 '🕌': 'joy',
 '➡': 'joy',
 '⛓': 'joy',
 '💻': 'joy',
 '🍋': 'joy',
 '☑': 'joy',
 '\U0001f929': 'joy',
 '🐋': 'joy',
 '📊': 'joy',
 '🏢': 'joy',
 '⬛': 'joy',
 '☎': 'joy',
 '📞': 'joy',
 '👒': 'joy',
 '💅': 'joy',
 '⛵': 'joy',
 '🗣': 'joy',
 '👢': 'joy',
 '🐖': 'joy',
 '🚩': 'joy',
 '🍸': 'joy',
 '🌸': 'joy',
 '⬆': 'joy',
 '👔': 'joy',
 '‼': 'joy',
 '\U0001f9db': 'joy',
 '🔮': 'joy',
 '💀': 'fear',
 '🌆': 'anticipation',
 '\U0001f94c': 'anticipation',
 '\U0001f9d8': 'joy',
 '🇪': 'anger',
 '🐫': 'joy',
 '💮': 'sadness',
 '☪': 'anticipation',
 '🌚': 'joy',
 '😐': 'sadness',
 '⛔': 'disgust',
 '🎎': 'joy',
 '⁉': 'disgust',
 '↖': 'trust',
 '🌈': 'trust',
 '♀': 'joy',
 '🚞': 'joy',
 '🐒': 'joy',
 '🔛': 'joy',
 '🖊': 'joy',
 '🕤': 'anticipation',
 '🚎': 'joy',
 '⛴': 'joy',
 '\U0001f954': 'joy',
 '\U0001f98c': 'joy',
 '🛩': 'joy',
 '\U0001f94b': 'trust',
 '\U0001f95b': 'joy',
 '㊙': 'joy',
 '🔋': 'joy',
 '▫': 'joy',
 '☸': 'joy',


In [17]:
list_replaced_emoji = []
for list_word in df_input.replace_tag:
    sentence_before = " ".join(list_word)
    sentence_after = ""
    for idx, char in enumerate(sentence_before):
#         if idx: 
#             print(char, sentence_before[idx-1], (char == sentence_before[idx-1]))
        if char in UNICODE_EMOJI :
            if not idx: sentence_after += dict_emoji_emotion[char]
            elif char != sentence_before[idx-1]: sentence_after += dict_emoji_emotion[char]
            else: continue
        else: sentence_after += char
    list_replaced_emoji.append(sentence_after.split())
df_input["replace_emoji"] = np.array(list_replaced_emoji)

df_input.head()

,hashtags,text,tweet_id,emotion,identification,split_to_word,replace_tag,replace_emoji
0,[Snapchat],"People who post ""add me on #Snapchat"" must be ...",0x376b20,anticipation,train,"[People, who, post, ""add, me, on, #Snapchat"", ...","[People, who, post, ""add, me, on, #Snapchat"", ...","[People, who, post, ""add, me, on, #Snapchat"", ..."
1,"[freepress, TrumpLegacy, CNN]","@brianklaas As we see, Trump is dangerous to #...",0x2d5350,sadness,train,"[@brianklaas, As, we, see,, Trump, is, dangero...","[[tag], As, we, see,, Trump, is, dangerous, to...","[[tag], As, we, see,, Trump, is, dangerous, to..."
3,[],Now ISSA is stalking Tasha 😂😂😂 <LH>,0x1cd5b0,fear,train,"[Now, ISSA, is, stalking, Tasha, 😂😂😂, <LH>]","[Now, ISSA, is, stalking, Tasha, 😂😂😂, <LH>]","[Now, ISSA, is, stalking, Tasha, joy, <LH>]"
5,"[authentic, LaughOutLoud]",@RISKshow @TheKevinAllison Thx for the BEST TI...,0x1d755c,joy,train,"[@RISKshow, @TheKevinAllison, Thx, for, the, B...","[[tag], [tag], Thx, for, the, BEST, TIME, toni...","[[tag], [tag], Thx, for, the, BEST, TIME, toni..."
6,[],Still waiting on those supplies Liscus. <LH>,0x2c91a8,anticipation,train,"[Still, waiting, on, those, supplies, Liscus.,...","[Still, waiting, on, those, supplies, Liscus.,...","[Still, waiting, on, those, supplies, Liscus.,..."


## III. Replace abbreviations

### setup abbreviation to sentence dictionary

In [18]:
dict_abbreviation = {}
with open("data/abbreviations.txt", 'r') as f_abbreviation:
    for idx, line in enumerate(f_abbreviation.readlines()):
        abbreviation, sentence = line.strip().split('=')
        dict_abbreviation[abbreviation] = sentence.split()
dict_abbreviation

{'7K': ['Sick'],
 ':-D': ['Laugher'],
 'A3': ['Anytime', 'Anywhere', 'Anyplace'],
 'AFAIK': ['As', 'Far', 'As', 'I', 'Know'],
 'AFK': ['Away', 'From', 'Keyboard'],
 'ASAP': ['As', 'Soon', 'As', 'Possible'],
 'ASL': ['Age', 'Sex', 'Location'],
 'ATK': ['At', 'The', 'Keyboard'],
 'ATM': ['At', 'The', 'Moment'],
 'B4': ['Before'],
 'B4N': ['Bye', 'For', 'Now'],
 'BAK': ['Back', 'At', 'Keyboard'],
 'BBL': ['Be', 'Back', 'Later'],
 'BBS': ['Be', 'Back', 'Soon'],
 'BFN': ['Bye', 'For', 'Now'],
 'BRB': ['Be', 'Right', 'Back'],
 'BRT': ['Be', 'Right', 'There'],
 'BTW': ['By', 'The', 'Way'],
 'CU': ['See', 'You'],
 'CUL8R': ['See', 'You', 'Later'],
 'CYA': ['See', 'You'],
 'FAQ': ['Frequently', 'Asked', 'Questions'],
 'FC': ['Fingers', 'Crossed'],
 'FWIW': ['For', 'What', "It's", 'Worth'],
 'FYI': ['For', 'Your', 'Information'],
 'G9': ['Genius'],
 'GAL': ['Get', 'A', 'Life'],
 'GG': ['Good', 'Game'],
 'GMTA': ['Great', 'Minds', 'Think', 'Alike'],
 'GN': ['Good', 'Night'],
 'GR8': ['Great'],
 '

In [19]:
df_input["replace_abbreviation"] = df_input.replace_emoji.apply(
    lambda list_word: 
    [word for list_word_after in [dict_abbreviation[word] if word in dict_abbreviation else [word] for word in list_word] for word in list_word_after]
)
df_input.head()

,hashtags,text,tweet_id,emotion,identification,split_to_word,replace_tag,replace_emoji,replace_abbreviation
0,[Snapchat],"People who post ""add me on #Snapchat"" must be ...",0x376b20,anticipation,train,"[People, who, post, ""add, me, on, #Snapchat"", ...","[People, who, post, ""add, me, on, #Snapchat"", ...","[People, who, post, ""add, me, on, #Snapchat"", ...","[People, who, post, ""add, me, on, #Snapchat"", ..."
1,"[freepress, TrumpLegacy, CNN]","@brianklaas As we see, Trump is dangerous to #...",0x2d5350,sadness,train,"[@brianklaas, As, we, see,, Trump, is, dangero...","[[tag], As, we, see,, Trump, is, dangerous, to...","[[tag], As, we, see,, Trump, is, dangerous, to...","[[tag], As, we, see,, Trump, is, dangerous, to..."
3,[],Now ISSA is stalking Tasha 😂😂😂 <LH>,0x1cd5b0,fear,train,"[Now, ISSA, is, stalking, Tasha, 😂😂😂, <LH>]","[Now, ISSA, is, stalking, Tasha, 😂😂😂, <LH>]","[Now, ISSA, is, stalking, Tasha, joy, <LH>]","[Now, ISSA, is, stalking, Tasha, joy, <LH>]"
5,"[authentic, LaughOutLoud]",@RISKshow @TheKevinAllison Thx for the BEST TI...,0x1d755c,joy,train,"[@RISKshow, @TheKevinAllison, Thx, for, the, B...","[[tag], [tag], Thx, for, the, BEST, TIME, toni...","[[tag], [tag], Thx, for, the, BEST, TIME, toni...","[[tag], [tag], Thx, for, the, BEST, TIME, toni..."
6,[],Still waiting on those supplies Liscus. <LH>,0x2c91a8,anticipation,train,"[Still, waiting, on, those, supplies, Liscus.,...","[Still, waiting, on, those, supplies, Liscus.,...","[Still, waiting, on, those, supplies, Liscus.,...","[Still, waiting, on, those, supplies, Liscus.,..."


## IV. Remove punctuation, LH and turn to lower

In [20]:
df_input["remove_punctuation"] = df_input.replace_abbreviation.apply(
    lambda list_word: 
    [word for word in [re.sub('[{}]'.format('!"#$%&\'()*+,-./:;<=>?[\\]^_`{|}~'), '', word.replace("<LH>", '').lower()) if word!="[tag]" else word for word in list_word ] if word]
)
df_input.head()

,hashtags,text,tweet_id,emotion,identification,split_to_word,replace_tag,replace_emoji,replace_abbreviation,remove_punctuation
0,[Snapchat],"People who post ""add me on #Snapchat"" must be ...",0x376b20,anticipation,train,"[People, who, post, ""add, me, on, #Snapchat"", ...","[People, who, post, ""add, me, on, #Snapchat"", ...","[People, who, post, ""add, me, on, #Snapchat"", ...","[People, who, post, ""add, me, on, #Snapchat"", ...","[people, who, post, add, me, on, snapchat, mus..."
1,"[freepress, TrumpLegacy, CNN]","@brianklaas As we see, Trump is dangerous to #...",0x2d5350,sadness,train,"[@brianklaas, As, we, see,, Trump, is, dangero...","[[tag], As, we, see,, Trump, is, dangerous, to...","[[tag], As, we, see,, Trump, is, dangerous, to...","[[tag], As, we, see,, Trump, is, dangerous, to...","[[tag], as, we, see, trump, is, dangerous, to,..."
3,[],Now ISSA is stalking Tasha 😂😂😂 <LH>,0x1cd5b0,fear,train,"[Now, ISSA, is, stalking, Tasha, 😂😂😂, <LH>]","[Now, ISSA, is, stalking, Tasha, 😂😂😂, <LH>]","[Now, ISSA, is, stalking, Tasha, joy, <LH>]","[Now, ISSA, is, stalking, Tasha, joy, <LH>]","[now, issa, is, stalking, tasha, joy]"
5,"[authentic, LaughOutLoud]",@RISKshow @TheKevinAllison Thx for the BEST TI...,0x1d755c,joy,train,"[@RISKshow, @TheKevinAllison, Thx, for, the, B...","[[tag], [tag], Thx, for, the, BEST, TIME, toni...","[[tag], [tag], Thx, for, the, BEST, TIME, toni...","[[tag], [tag], Thx, for, the, BEST, TIME, toni...","[[tag], [tag], thx, for, the, best, time, toni..."
6,[],Still waiting on those supplies Liscus. <LH>,0x2c91a8,anticipation,train,"[Still, waiting, on, those, supplies, Liscus.,...","[Still, waiting, on, those, supplies, Liscus.,...","[Still, waiting, on, those, supplies, Liscus.,...","[Still, waiting, on, those, supplies, Liscus.,...","[still, waiting, on, those, supplies, liscus]"


## V. Lemmatization

In [25]:
def get_wordnet_pos(treebank_tag):
    if treebank_tag.startswith('J'):
        return wordnet.ADJ
    elif treebank_tag.startswith('V'):
        return wordnet.VERB
    elif treebank_tag.startswith('N'):
        return wordnet.NOUN
    elif treebank_tag.startswith('R'):
        return wordnet.ADV
    else:
        return None


def lemmatize_sentence(list_word):
    res = []
    lemmatizer = WordNetLemmatizer()
    for word, pos in pos_tag(list_word):
        wordnet_pos = get_wordnet_pos(pos) or wordnet.NOUN
        res.append(lemmatizer.lemmatize(word, pos=wordnet_pos))

    return res

# lambda x: ' '.join([lmtzr.lemmatize(word,'v') for word in x.split()])
df_input["lemmatization"] = df_input.remove_punctuation.apply(
    lambda list_word: 
    lemmatize_sentence(list_word)
)
df_input.head()

,hashtags,text,tweet_id,emotion,identification,split_to_word,replace_tag,replace_emoji,replace_abbreviation,remove_punctuation,lemmatization,remove_stopword
0,[Snapchat],"People who post ""add me on #Snapchat"" must be ...",0x376b20,anticipation,train,"[People, who, post, ""add, me, on, #Snapchat"", ...","[People, who, post, ""add, me, on, #Snapchat"", ...","[People, who, post, ""add, me, on, #Snapchat"", ...","[People, who, post, ""add, me, on, #Snapchat"", ...","[people, who, post, add, me, on, snapchat, mus...","[people, who, post, add, me, on, snapchat, mus...","[[people, who, post, add, me, on, snapchat, mu..."
1,"[freepress, TrumpLegacy, CNN]","@brianklaas As we see, Trump is dangerous to #...",0x2d5350,sadness,train,"[@brianklaas, As, we, see,, Trump, is, dangero...","[[tag], As, we, see,, Trump, is, dangerous, to...","[[tag], As, we, see,, Trump, is, dangerous, to...","[[tag], As, we, see,, Trump, is, dangerous, to...","[[tag], as, we, see, trump, is, dangerous, to,...","[[tag], a, we, see, trump, be, dangerous, to, ...","[[[tag], a, we, see, trump, be, dangerous, to,..."
3,[],Now ISSA is stalking Tasha 😂😂😂 <LH>,0x1cd5b0,fear,train,"[Now, ISSA, is, stalking, Tasha, 😂😂😂, <LH>]","[Now, ISSA, is, stalking, Tasha, 😂😂😂, <LH>]","[Now, ISSA, is, stalking, Tasha, joy, <LH>]","[Now, ISSA, is, stalking, Tasha, joy, <LH>]","[now, issa, is, stalking, tasha, joy]","[now, issa, be, stalk, tasha, joy]","[[now, issa, be, stalk, tasha, joy]]"
5,"[authentic, LaughOutLoud]",@RISKshow @TheKevinAllison Thx for the BEST TI...,0x1d755c,joy,train,"[@RISKshow, @TheKevinAllison, Thx, for, the, B...","[[tag], [tag], Thx, for, the, BEST, TIME, toni...","[[tag], [tag], Thx, for, the, BEST, TIME, toni...","[[tag], [tag], Thx, for, the, BEST, TIME, toni...","[[tag], [tag], thx, for, the, best, time, toni...","[[tag], [tag], thx, for, the, best, time, toni...","[[[tag], [tag], thx, for, the, best, time, ton..."
6,[],Still waiting on those supplies Liscus. <LH>,0x2c91a8,anticipation,train,"[Still, waiting, on, those, supplies, Liscus.,...","[Still, waiting, on, those, supplies, Liscus.,...","[Still, waiting, on, those, supplies, Liscus.,...","[Still, waiting, on, those, supplies, Liscus.,...","[still, waiting, on, those, supplies, liscus]","[still, wait, on, those, supply, liscus]","[[still, wait, on, those, supply, liscus]]"


## VI. Remove stopword

### setup stopword with tweet

In [22]:
with open("data/stopword_all.txt", 'r') as f_stopword:
    list_word = stopwords.words('english')
    for line in f_stopword.readlines():
#         print(line.strip().split(','))
        list_word.extend(line.strip().split(','))
    list_word = list(set(list_word))

with open("data/stopword.txt", 'w') as f_stopword:
    f_stopword.write("\n".join(list_word))

### read stopword file

In [23]:
with open("data/stopword.txt", 'r') as f_stopword:
    list_stopword = []
    for line in f_stopword.readlines():
        list_stopword.append(line.strip())
    
list_stopword

['broken',
 'rice',
 '',
 'recent',
 'uploaded',
 'viv',
 'created',
 'builds',
 'bloggers',
 'requests',
 'couple',
 'enables',
 'ordering',
 'eaten',
 'dinner',
 'rental',
 'money',
 'byeeee',
 'serves',
 "mustn't",
 'online',
 'courses',
 'whatev',
 'newly',
 'whaow',
 'events',
 'dear',
 'neways',
 "should've",
 'gift',
 'programs',
 'updates',
 'lols',
 'nooo',
 'tired',
 'doubles',
 'ymmv',
 'shops',
 'waited',
 'feels',
 'com',
 'shouldn',
 'morning',
 'distract',
 'excluded',
 'recovers',
 'writes',
 'occured',
 'yhoo',
 'gave',
 'treated',
 '.com',
 'homeworks',
 'upgrade',
 'afaik',
 'projects',
 'issue',
 'albums',
 'isn',
 'catch',
 'hee',
 'btw',
 'box',
 'february',
 'is',
 'singled',
 'hehehe',
 'safe',
 'produces',
 'approve',
 'wahaha',
 'buys',
 'dont',
 '#followme',
 'furnish',
 'does',
 'sms',
 'loaned',
 'primary',
 'june',
 'refuses',
 'talked',
 'whom',
 'removes',
 'dated',
 'duper',
 'condominium',
 'huge',
 'obvious',
 'tbh',
 'woohoo',
 'runs',
 'yeah',
 'com

In [26]:
df_input["remove_stopword"] = df_input.lemmatization.apply(
    lambda list_word: 
    [word for word in list_word if word not in list_stopword]
)
df_input.head()

,hashtags,text,tweet_id,emotion,identification,split_to_word,replace_tag,replace_emoji,replace_abbreviation,remove_punctuation,lemmatization,remove_stopword
0,[Snapchat],"People who post ""add me on #Snapchat"" must be ...",0x376b20,anticipation,train,"[People, who, post, ""add, me, on, #Snapchat"", ...","[People, who, post, ""add, me, on, #Snapchat"", ...","[People, who, post, ""add, me, on, #Snapchat"", ...","[People, who, post, ""add, me, on, #Snapchat"", ...","[people, who, post, add, me, on, snapchat, mus...","[people, who, post, add, me, on, snapchat, mus...","[snapchat, must, dehydrate, thats]"
1,"[freepress, TrumpLegacy, CNN]","@brianklaas As we see, Trump is dangerous to #...",0x2d5350,sadness,train,"[@brianklaas, As, we, see,, Trump, is, dangero...","[[tag], As, we, see,, Trump, is, dangerous, to...","[[tag], As, we, see,, Trump, is, dangerous, to...","[[tag], As, we, see,, Trump, is, dangerous, to...","[[tag], as, we, see, trump, is, dangerous, to,...","[[tag], a, we, see, trump, be, dangerous, to, ...","[[tag], see, trump, dangerous, freepress, arou..."
3,[],Now ISSA is stalking Tasha 😂😂😂 <LH>,0x1cd5b0,fear,train,"[Now, ISSA, is, stalking, Tasha, 😂😂😂, <LH>]","[Now, ISSA, is, stalking, Tasha, 😂😂😂, <LH>]","[Now, ISSA, is, stalking, Tasha, joy, <LH>]","[Now, ISSA, is, stalking, Tasha, joy, <LH>]","[now, issa, is, stalking, tasha, joy]","[now, issa, be, stalk, tasha, joy]","[issa, stalk, tasha, joy]"
5,"[authentic, LaughOutLoud]",@RISKshow @TheKevinAllison Thx for the BEST TI...,0x1d755c,joy,train,"[@RISKshow, @TheKevinAllison, Thx, for, the, B...","[[tag], [tag], Thx, for, the, BEST, TIME, toni...","[[tag], [tag], Thx, for, the, BEST, TIME, toni...","[[tag], [tag], Thx, for, the, BEST, TIME, toni...","[[tag], [tag], thx, for, the, best, time, toni...","[[tag], [tag], thx, for, the, best, time, toni...","[[tag], [tag], thx, best, heartbreakingly, aut..."
6,[],Still waiting on those supplies Liscus. <LH>,0x2c91a8,anticipation,train,"[Still, waiting, on, those, supplies, Liscus.,...","[Still, waiting, on, those, supplies, Liscus.,...","[Still, waiting, on, those, supplies, Liscus.,...","[Still, waiting, on, those, supplies, Liscus.,...","[still, waiting, on, those, supplies, liscus]","[still, wait, on, those, supply, liscus]","[still, supply, liscus]"


## Save to Pickle after Preprocssing

In [27]:
df_input.to_pickle("data/train_df_han_preprocess.pkl") 

## Test Preprocessing

In [ ]:
from data.unicode_codes import EMOJI_UNICODE, UNICODE_EMOJI

list_cal_emoji = df_input.replace_tag.apply(
    lambda list_word: 
    [char for char in " ".join(list_word) if char in UNICODE_EMOJI]
)

emotion_array = ['joy', 'sadness', 'trust', 'anticipation', 'fear', 'anger', 'disgust', 'surprise']
dict_emoji_all_emotion = defaultdict(list)
dict_emoji_emotion = {}
seties_emotion = df_input.emotion
for idx, list_emoji in enumerate(list_cal_emoji): 
    list_emoji = list(set(list_emoji))  
    for emoji in list_emoji:
        if not dict_emoji_all_emotion[emoji]: dict_emoji_all_emotion[emoji]=[0]*len(emotion_array)
        dict_emoji_all_emotion[emoji][emotion_array.index(seties_emotion.iloc[idx])] += 1

for emoji, list_emoji_num in dict_emoji_all_emotion.items():
    dict_emoji_emotion[emoji] = emotion_array[np.argmax(list_emoji_num)]

dict_emoji_emotion

In [ ]:
dict_abbreviation = {}
with open("data/abbreviations.txt", 'r') as f_abbreviation:
    for idx, line in enumerate(f_abbreviation.readlines()):
        abbreviation, sentence = line.strip().split('=')
        dict_abbreviation[abbreviation] = sentence.split()
dict_abbreviation

In [ ]:
def get_wordnet_pos(treebank_tag):
    if treebank_tag.startswith('J'):
        return wordnet.ADJ
    elif treebank_tag.startswith('V'):
        return wordnet.VERB
    elif treebank_tag.startswith('N'):
        return wordnet.NOUN
    elif treebank_tag.startswith('R'):
        return wordnet.ADV
    else:
        return None


def lemmatize_sentence(list_word):
    res = []
    lemmatizer = WordNetLemmatizer()
    for word, pos in pos_tag(list_word):
        wordnet_pos = get_wordnet_pos(pos) or wordnet.NOUN
        res.append(lemmatizer.lemmatize(word, pos=wordnet_pos))

    return res


In [ ]:
with open("data/stopword.txt", 'r') as f_stopword:
    list_stopword = []
    for line in f_stopword.readlines():
        list_stopword.append(line.strip())

In [35]:
def preprocessing(name, df, dict_emoji_emotion, dict_abbreviation, list_stopword):
    df["preprocessing"] = df["text"].str.split()
    
    df["preprocessing"] = df["preprocessing"].apply(
        lambda list_word: 
        ["[tag]" if word[0]=='@' else word for word in list_word]
    )
    
    list_replaced_emoji = []
    for list_word in df.preprocessing:
        sentence_before = " ".join(list_word)
        sentence_after = ""
        for idx, char in enumerate(sentence_before):
    #         if idx: 
    #             print(char, sentence_before[idx-1], (char == sentence_before[idx-1]))
            if char in UNICODE_EMOJI :
                emotion = dict_emoji_emotion.get(char)
                if not emotion: continue
                if not idx: sentence_after += emotion
                elif char != sentence_before[idx-1]: 
                    sentence_after += emotion
                else: continue
            else: sentence_after += char
        list_replaced_emoji.append(sentence_after.split())
    df["preprocessing"] = np.array(list_replaced_emoji)
    
    df["preprocessing"] = df["preprocessing"].apply(
        lambda list_word: 
        [word for list_word_after in [dict_abbreviation[word] if word in dict_abbreviation else [word] for word in list_word] for word in list_word_after]
    )
    
    df["preprocessing"] = df["preprocessing"].apply(
        lambda list_word: 
        [word for word in [re.sub('[{}]'.format('!"#$%&\'()*+,-./:;<=>?[\\]^_`{|}~'), '', word.replace("<LH>", '').lower()) if word!="[tag]" else word for word in list_word ] if word]
    )
    
    lmtzr = WordNetLemmatizer()
    df["preprocessing"] = df["preprocessing"].apply(
        lambda list_word: 
        lemmatize_sentence(list_word)
    )
    
    df["preprocessing"] = df["preprocessing"].apply(
        lambda list_word: 
        [word for word in list_word if word not in list_stopword]
    )

    df.to_pickle("data/{}.pkl".format(name)) 


In [37]:
preprocessing("test_df_han_preprocess", test_df, dict_emoji_emotion, dict_abbreviation, list_stopword)

# Load Pickle

In [ ]:
## load a pickle file
trains = pd.read_pickle("data/train_df_han_preprocess.pkl")
tests = pd.read_pickle("data/test_df_han_preprocess.pkl")

# Import Package for BOW

In [ ]:
import pandas as pd
import numpy as np
import json
import re, csv
import nltk
import string

from pandas.io.json import json_normalize
from nltk.corpus import stopwords
from nltk import word_tokenize
from gensim.models import KeyedVectors   
from nltk.tokenize import TweetTokenizer
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer

from keras.models import Model
from keras.layers import Input, Dense
from keras.layers import ReLU, Softmax

# Set Tfidf Vectorizer

In [ ]:
tknzr = TweetTokenizer()
# TF-IDF

tfidf_vectorizer = TfidfVectorizer(
    max_features=5000, 
    use_idf=True, 
    tokenizer=tknzr.tokenize,
    stop_words=stopwords.words('english'))

tfidf_vectorizer.fit(trains['remove_stopword'])

# Seperate Data to Train and Validation

In [ ]:
sep = int(trains.shape[0] * 0.8) # 20% for validation

BOW_train = trains.remove_stopword[:sep]
BOW_train_label = trains.emotion[:sep]
BOW_val = trains.remove_stopword[sep:]
BOW_val_label = trains.emotion[sep:]

# Convert to Input Data

In [ ]:
X_train = tfidf_vectorizer.transform(BOW_train)
y_train = BOW_train_label

X_val = tfidf_vectorizer.transform(BOW_val)
y_val = BOW_val_label

# Set Up Emoji List for Encode / Decode

In [ ]:
emotion_array = ['joy', 'sadness', 'trust', 'anticipation', 'fear', 'anger', 'disgust', 'surprise']

In [ ]:
def emotion_to_sequence(emotions):
    tokenized_emotions = []
    for string in emotions:
        token = emotion_array.index(string)
        tokenized_emotions.append(token)
    return to_categorical(tokenized_emotions)

In [ ]:
y_train = emotion_to_sequence(y_train)
y_val = emotion_to_sequence(y_val)

# BOW Model Set Up

In [ ]:
DENSE1_DIM = 256
DENSE2_DIM = 256
DENSE3_DIM = 64
CATEGORY_NUM = 8
BATCH_SIZE = 128
VALIDATION_SPLIT = 0.2
EPOCHS = 40
DROPOUT_DENSE = 0.2

from datetime import datetime 

TIME_NOW = datetime.now().isoformat().split('.')[0]

#############################################################################################
MODEL_DESCRIPTION = "{}_{}".format("BOW", "ThreeHidden")
#############################################################################################

csv_logger = CSVLogger(
    'logs/{}_{}.csv'.format(MODEL_DESCRIPTION, TIME_NOW))

SUMMARY_REPORT_PATH = 'summary/{}_{}.txt'.format(MODEL_DESCRIPTION, TIME_NOW)
                    

def model_setting(texts, labels, texts_val, labels_val):
    #######################################
    # build up the model 
    # input: texts (np), labels (np)
    # output: callback
    #######################################
    
    shuffle_indices = np.random.permutation(texts.shape[0],)
    texts = texts[shuffle_indices]
    labels = labels[shuffle_indices]
    
    # model build
    model = Sequential()
    model.add(Dense(DENSE1_DIM, kernel_initializer='normal', activation='relu'))
    model.add(Dropout(DROPOUT_DENSE))
    model.add(Dense(DENSE2_DIM, kernel_initializer='normal', activation='relu'))
    model.add(Dropout(DROPOUT_DENSE))
    model.add(Dense(DENSE3_DIM, kernel_initializer='normal', activation='relu'))
    model.add(Dropout(DROPOUT_DENSE))
    
    model.add(Dense(CATEGORY_NUM, kernel_initializer='normal', activation='softmax'))
    model.compile(loss="categorical_crossentropy", optimizer="nadam", metrics=["accuracy"])
    
    model.summary()
    with open(SUMMARY_REPORT_PATH, mode='w') as file_summary:
        model.summary(print_fn=lambda line: file_summary.write(line+'\n'))
        
    history_callback = model.fit(
        texts_train, labels_train, 
        epochs=EPOCHS, batch_size=BATCH_SIZE, 
        callbacks=[csv_logger],
        verbose=1, 
        shuffle=True, 
        validation_data=(texts_val, labels_val))
    
    model.save('model/{}_{}.h5'.format(MODEL_DESCRIPTION,TIME_NOW))
    
    return history_callback

history_callback = model_setting(X_train, y_train, X_val, y_val)

# Import Package for Model Setting

In [ ]:
import pandas as pd
import numpy as np
import json
import csv
from nltk.corpus import stopwords
import nltk
import string

from nltk import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.tokenize import TweetTokenizer

from gensim.models import KeyedVectors   
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer

import keras
from keras.models import Model, Sequential, load_model
from keras.layers import Input, Embedding, Dense, Bidirectional, LSTM, GRU, Dropout
from keras.callbacks import CSVLogger

from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer
from keras.utils import to_categorical
from keras.initializers import Constant

# Text to Sequence

In [ ]:
ps = PorterStemmer()
lmtzr = WordNetLemmatizer()

MAX_SEQUENCE_LENGTH = 40

def text_to_sequence(list_list_word):
    tokenized_texts = []
    for list_word in list_list_word:
        tokenized_text = []
        for idx, word in enumerate(list_word):
            
            # truncate
            if idx >= MAX_SEQUENCE_LENGTH: break 
                
            try:
                token = token_dict[word.lower()]
            except:
                token = MAX_WORD
                
            tokenized_text.append(token)
            
        # padding 
        if len(tokenized_text) < MAX_SEQUENCE_LENGTH: 
            tokenized_text.extend([MAX_WORD]*(MAX_SEQUENCE_LENGTH-len(tokenized_text)))
                
        tokenized_texts.append(tokenized_text)
    
    return np.array(tokenized_texts)

# Set Tfidf Vectorizer

In [ ]:
# TF-IDF
MAX_WORD = 15000
tfidf_vectorizer = TfidfVectorizer(
    max_features=MAX_WORD, 
    use_idf=True, 
    )
tfidf_vectorizer.fit(trains['remove_stopword'])

In [ ]:
token_dict = {}
for idx, token in enumerate(tfidf_vectorizer.get_feature_names()):
    token_dict[token] = idx

# Load Word2Vec Model and Setup Embedding Matrix

In [ ]:
word2vec_model = KeyedVectors.load_word2vec_format(
    "data/model_ENC3.bin", binary=True)

In [ ]:
EMBEDDING_DIM = 300
def word2vec_matrix():
    matrix_emb = np.zeros((len(token_dict)+1, EMBEDDING_DIM)) # replace MAX_WORD to len(token_dict)
    for word, idx in token_dict.items():
        try:
            vector =  word2vec_model.wv[word]
        except:
            vector = np.zeros(300,)
        if idx < MAX_WORD:
            matrix_emb[idx] = np.array(vector)
    matrix_emb[MAX_WORD] = np.zeros(300,)     
    return matrix_emb

# Setup Emoji List for Encode / Decode

In [ ]:
emotion_array = ['joy', 'sadness', 'trust', 'anticipation', 'fear', 'anger', 'disgust', 'surprise']

In [ ]:
def emotion_to_sequence(emotions):
    tokenized_emotions = []
    for string in emotions:
        token = emotion_array.index(string)
        tokenized_emotions.append(token)
    return to_categorical(tokenized_emotions)

In [ ]:
train_sequence = text_to_sequence(trains['remove_stopword'])
train_label = emotion_to_sequence(trains['emotion'])

# Setup Model 

In [ ]:
LSTM1_DIM = 256
DENSE1_DIM = 256
DENSE2_DIM = 256
DENSE3_DIM = 64
CATEGORY_NUM = 8
BATCH_SIZE = 128
VALIDATION_SPLIT = 0.2
EPOCHS = 7
DROPOUT_LSTM = 0.2
DROPOUT_DENSE = 0.2



from datetime import datetime 

TIME_NOW = datetime.now().isoformat().split('.')[0]

#############################################################################################
MODEL_DESCRIPTION = "{}_{}_{}".format("BiGRU_TwoHidden", str(LSTM1_DIM), str(DROPOUT_LSTM))
#############################################################################################

csv_logger = CSVLogger(
    'logs/{}_{}.csv'.format(MODEL_DESCRIPTION, TIME_NOW))

SUMMARY_REPORT_PATH = 'summary/{}_{}.txt'.format(MODEL_DESCRIPTION, TIME_NOW)
                    

def model_setting(texts, labels, matrix_emb="uniform"):
    #######################################
    # build up the model 
    # input: texts (np), labels (np), embedding matrix (np)
    # output: callback
    #######################################
    
    import numpy as np
    import pandas as pd
    
    if matrix_emb != "uniform": matrix_emb = Constant(matrix_emb)
        
    shuffle_indices = np.random.permutation(texts.shape[0],)
    texts = texts[shuffle_indices]
    labels = labels[shuffle_indices]
        
    #  split data to train & validation
    validation_size = int(texts.shape[0]*VALIDATION_SPLIT)
    texts_train = texts[:-validation_size]
    texts_val = texts[-validation_size:]
    labels_train = labels[:-validation_size]
    labels_val = labels[-validation_size:]
    
    # model build
    model = Sequential()
    model.add(Embedding(MAX_WORD+1, EMBEDDING_DIM, embeddings_initializer=matrix_emb, input_length=MAX_SEQUENCE_LENGTH, trainable=True)) # with embedding matrix
    model.add(GRU(LSTM1_DIM, dropout=DROPOUT_LSTM, recurrent_dropout=DROPOUT_LSTM))

    model.add(Dense(DENSE1_DIM, kernel_initializer='normal', activation='relu'))
    model.add(Dropout(DROPOUT_DENSE))
    model.add(Dense(DENSE2_DIM, kernel_initializer='normal', activation='relu'))
    model.add(Dropout(DROPOUT_DENSE))
    
    model.add(Dense(CATEGORY_NUM, kernel_initializer='normal', activation='softmax'))
    model.compile(loss="categorical_crossentropy", optimizer="nadam", metrics=["accuracy"])
    
    model.summary()
    with open(SUMMARY_REPORT_PATH, mode='w') as file_summary:
        model.summary(print_fn=lambda line: file_summary.write(line+'\n'))
        
    history_callback = model.fit(
        texts_train, labels_train, 
        epochs=EPOCHS, batch_size=BATCH_SIZE, 
        callbacks=[csv_logger],
        verbose=1, 
        shuffle=True, 
        validation_data=(texts_val, labels_val))
    
    model.save('model/{}_{}.h5'.format(MODEL_DESCRIPTION,TIME_NOW))
    
    return history_callback

In [ ]:
history_callback = model_setting(train_sequence, train_label, word2vec_matrix())

# Plot Loss and Accuracy of Train and Validation

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

def show_train_history(train_history, train, validation):
    plt.plot(train_history.history[train])
    plt.plot(train_history.history[validation])
    plt.title('Train History')
    plt.ylabel(train)
    plt.xlabel('Epoch')
    plt.legend(['train', 'validation'], loc='upper left')
    plt.show()

df_his_acc = pd.DataFrame({'train_acc':history_callback.history['acc'], 'validation_acc':history_callback.history['val_acc']})
show_train_history(history_callback, 'acc', 'val_acc')

In [ ]:
def show_train_history(train_history, train, validation):
    plt.plot(train_history.history[train])
    plt.plot(train_history.history[validation])
    plt.title('Train History')
    plt.ylabel(train)
    plt.xlabel('Epoch')
    plt.legend(['train', 'validation'], loc='upper left')
    plt.show()
df_his_acc = pd.DataFrame({'train_loss':history_callback.history['loss'], 'validation_loss':history_callback.history['val_loss']})
show_train_history(history_callback, 'loss', 'val_loss')

# Predict Test Data

In [ ]:
## predict
tests = pd.read_pickle("data/train_df_han_preprocess.pkl")
X_test = text_to_sequence(tests['text'])
model = load_model("model/{}_{}.h5".format(MODEL_DESCRIPTION, TIME_NOW))

pred_result = model.predict(X_test, batch_size=128)
pred_result = np.array(
    [emotion_array[pred_result_idx] for pred_result_idx in np.argmax(pred_result, axis=1)])

tests['emotion'] = pred_result
tests = tests.drop('hashtags', axis=1)
tests = tests.drop('text', axis=1)
tests = tests.drop('identification', axis=1)
tests.rename(columns={'tweet_id':'id'}, inplace=True)
tests.to_csv('predict_result/prediction_{}_{}.csv'.format(MODEL_DESCRIPTION, TIME_NOW), index=False)